<a href="https://colab.research.google.com/github/rsadiq/DFI/blob/master/DFI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


1.   Git CLone the DFI Repo
2.   Download the pretrained weights
3.   Move weights to pretrained directory







In [1]:
!git clone https://github.com/backseason/DFI.git
!cd DFI/


Cloning into 'DFI'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 42 (delta 6), reused 33 (delta 4), pack-reused 0
Unpacking objects: 100% (42/42), done.


In [ ]:
import os
os.chdir('DFI')
!gdown --id 1N29cJghKKJOHbKgpwR2_Ui64umCE-XG3

In [3]:
!mkdir pretrained
!mv dfi.pth pretrained/dfi.pth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [26]:
os.mkdir('/content/DFI/Everest')
os.mkdir('/content/DFI/Everest/input_images')

In [ ]:
!unzip "/content/drive/My Drive/Everest/ing_hashtag_search_RO.zip" "-d" "/content/DFI/Everest/input_images"

**ResNet Block**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, stride=stride, bias=False) 
        self.bn1 = nn.BatchNorm2d(planes)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1,
                               padding=dilation, dilation=dilation, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        for i in self.bn2.parameters():
            i.requires_grad = False
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        for i in self.bn3.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers):
        self.freeze_bn = True
        self.inplanes = 64
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        for i in self.bn1.parameters():
            i.requires_grad = False
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, ceil_mode=True)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=1, dilation=2)

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion or dilation == 2 or dilation == 4:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )
        for i in downsample._modules['1'].parameters():
            i.requires_grad = False
        layers = []
        layers.append(block(self.inplanes, planes, stride,dilation=dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes,dilation=dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        tmp_x = []
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        tmp_x.append(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        tmp_x.append(x)
        x = self.layer2(x)
        tmp_x.append(x)
        x = self.layer3(x)
        tmp_x.append(x)
        x = self.layer4(x)
        tmp_x.append(x)

        return tmp_x


class ResNet_PPM(nn.Module):
    def __init__(self, block, layers):
        super(ResNet_PPM,self).__init__()
        self.resnet = ResNet(block, layers)

        self.in_planes = 256

        self.ppm_pre = nn.Sequential(
            nn.Conv2d(2048, self.in_planes, 1, 1, bias=False),
            nn.GroupNorm(32, self.in_planes),
        )
        ppms = []
        for ii in [1, 3, 5]:
            ppms.append(nn.Sequential(
                nn.AdaptiveAvgPool2d(ii), 
                nn.Conv2d(self.in_planes, self.in_planes, 1, 1, bias=False), 
                nn.GroupNorm(32, self.in_planes),
                ))
        self.ppms = nn.ModuleList(ppms)

        self.ppm_cat = nn.Sequential(
            nn.Conv2d(self.in_planes, self.in_planes, 3, 1, 1, bias=False),
            nn.GroupNorm(32, self.in_planes),
        )
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.resnet(x)

        x_pre = self.ppm_pre(x[-1])
        x_ppm = x_pre
        for k in range(len(self.ppms)):
            x_ppm = torch.add(x_ppm, F.interpolate(self.ppms[k](x_pre), x_pre.size()[2:], mode='bilinear', align_corners=True))
        x_ppm = self.ppm_cat(self.relu(x_ppm))
        x.append(x_ppm)

        return x

def resnet50_ppm():
    model = ResNet_PPM(Bottleneck, [3, 4, 6, 3])
    return model


**DFI BLOCK**

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

from .resnet import resnet50_ppm

config = {'converter': [[64,256,512,1024,2048,256],[32,64,128,256,256,256]], 
          'dfims': [[[32,64,128,256,256,256], 32, 0], [[32,64,128,256,256,256], 64, 1], [[32,64,128,256,256,256], 128, 2], [[32,64,128,256,256,256], 256, 3]], 
          'dfims_id': [[0,1,2,3,4,5], [0,1,2,3,4,5], [0,1,2,3,4,5], [0,1,2,3,4,5]], 
          'tams': [32, 64, 128, 256],
          'predictors': [[[32, 64, 128, 256], True], [[32, 64, 128, 256], False], [[32, 64, 128, 256], True]], 
          'predictors_id': [0,1,2,3] } 

def gn(planes, channel_per_group=4, max_groups=32):
    groups = planes // channel_per_group
    return nn.GroupNorm(min(groups, max_groups), planes)

class Converter(nn.Module):
    def __init__(self, list_k):
        super(Converter, self).__init__()
        up = []
        for i in range(len(list_k[0])):
            up.append(nn.Sequential(
                nn.Conv2d(list_k[0][i], list_k[1][i], 1, 1, bias=False), 
                gn(list_k[1][i]), 
                nn.ReLU(inplace=True),
                ))
        self.convert = nn.ModuleList(up)

    def forward(self, x):
        out = []
        for i in range(len(x)):
            out.append(self.convert[i](x[i]))
        return out

class DFIM(nn.Module): 
    def __init__(self, list_k, k, size_id, modes=3):
        super(DFIM, self).__init__()
        self.len = len(list_k)
        self.size_id = size_id
        up = []
        for i in range(len(list_k)):
            up.append(nn.Sequential(nn.Conv2d(list_k[i], k, 1, 1, bias=False), gn(k)))
        self.merge = nn.ModuleList(up)
        merge_convs, fcs, convs = [], [], []
        for m in range(modes):
            merge_convs.append(nn.Sequential(
                        nn.Conv2d(k, k//4, 1, 1, bias=False), 
                        gn(k//4), 
                        nn.ReLU(inplace=True),
                        nn.Conv2d(k//4, k, 1, 1, bias=False),
                        gn(k),
                    ))
            fcs.append(nn.Sequential(
                    nn.Linear(k, k // 4, bias=False),
                    nn.ReLU(inplace=True),
                    nn.Linear(k // 4, self.len, bias=False),
                ))
            convs.append(nn.Sequential(nn.Conv2d(k, k, 3, 1, 1, bias=False), gn(k), nn.ReLU(inplace=True)))
        self.merge_convs = nn.ModuleList(merge_convs)
        self.fcs = nn.ModuleList(fcs)
        self.convs = nn.ModuleList(convs)
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.softmax = nn.Softmax(dim=1)
        self.relu =nn.ReLU(inplace=True)

    def forward(self, list_x, mode=3):
        x_size = list_x[self.size_id].size()
        feas = []
        for i in range(len(list_x)):
            feas.append(self.merge[i](F.interpolate(list_x[i], x_size[2:], mode='bilinear', align_corners=True)).unsqueeze(dim=1))
        feas = torch.cat(feas, dim=1) # Nx6xCxHxW
        fea_sum = torch.sum(feas, dim=1) # NxCxHxW

        if mode == 3:
            outs = []
            for mode_ in range(3):
                fea_u = self.merge_convs[mode_](fea_sum)
                fea_s = self.gap(fea_u).squeeze(-1).squeeze(-1) # NxC
                fea_z = self.fcs[mode_](fea_s) # Nx6
                selects = self.softmax(fea_z) # Nx6
                feas_f = selects.reshape(x_size[0], self.len, 1, 1, 1).expand_as(feas) * feas # Nx6xCxHxW
                _, index = torch.topk(selects, 3, dim=1) # Nx3
                selected = []
                for i in range(x_size[0]):
                    selected.append(torch.index_select(feas_f, dim=1, index=index[i]))
                selected = torch.cat(selected, dim=0)
                fea_v = selected.sum(dim=1)
                outs.append(self.convs[mode_](self.relu(fea_v)))
            return torch.cat(outs, dim=0)
        else:
            fea_u = self.merge_convs[mode](fea_sum)
            fea_s = self.gap(fea_u).squeeze(-1).squeeze(-1) # NxC
            fea_z = self.fcs[mode](fea_s) # Nx6
            selects = self.softmax(fea_z) # Nx6
            feas_f = selects.reshape(x_size[0], self.len, 1, 1, 1).expand_as(feas) * feas # Nx6xCxHxW
            _, index = torch.topk(selects, 3, dim=1) # Nx3
            selected = []
            for i in range(x_size[0]):
                selected.append(torch.index_select(feas_f, dim=1, index=index[i]))
            selected = torch.cat(selected, dim=0)
            fea_v = selected.sum(dim=1)
            return self.convs[mode](self.relu(fea_v))

class TAM(nn.Module): # TAM
    reduction = 4
    def __init__(self, k):
        super(TAM, self).__init__()
        k_mid = int(k // self.reduction)
        self.attention = nn.Sequential(
            nn.Conv2d(k, k_mid, 1, 1, bias=False),
            gn(k_mid),
            nn.ReLU(inplace=True),
            nn.Conv2d(k_mid, k, 1, 1, bias=False),
            gn(k),
            nn.Sigmoid(),
        )
        self.block = nn.Sequential(nn.Conv2d(k, k, 3, 1, 1, bias=False), gn(k), nn.ReLU(inplace=True))

    def forward(self, x):
        out = self.attention(x)
        out = torch.add(x, torch.mul(x, out))
        out = self.block(out)
        return out

class Predictor(nn.Module):
    def __init__(self, list_k, deep_sup):
        super(Predictor, self).__init__()
        self.trans = nn.ModuleList()
        for i in range(len(list_k)):
            self.trans.append(nn.Conv2d(list_k[i], 1, 1, 1))
        self.fuse = nn.Conv2d(len(list_k), 1, 1, 1)
        self.deep_sup = deep_sup

    def forward(self, list_x, x_size=None):
        up_x = []
        for i, i_x in enumerate(list_x):
            up_x.append(F.interpolate(self.trans[i](i_x), x_size[2:], mode='bilinear', align_corners=True))
        fuse = self.fuse(torch.cat(up_x, dim = 1))
        if self.deep_sup:
            return [fuse, up_x]
        else:
            return [fuse]

def extra_layer(base):
    converter, dfims, tams, predictors = [], [], [], []
    converter = Converter(config['converter'])

    for k in config['dfims']:
        dfims += [DFIM(k[0], k[1], k[2])]

    for k in config['tams']:
        tams += [TAM(k)]

    for k in config['predictors']:
        predictors += [Predictor(k[0], k[1])]

    return base, converter, dfims, tams, predictors


class DFI(nn.Module):
    def __init__(self, base, converter, dfims, tams, predictors):
        super(DFI, self).__init__()
        self.dfims_id = config['dfims_id']
        self.predictors_id = config['predictors_id']

        self.base = base
        self.converter = converter
        self.dfims = nn.ModuleList(dfims)
        self.tams = nn.ModuleList(tams)
        self.predictors = nn.ModuleList(predictors)

    def forward(self, x, mode = 3):
        x_size = x.size()
        x = self.converter(self.base(x))

        # DFIM
        dfims = []
        for k in range(len(self.dfims)):
           dfims.append(self.dfims[k]([x[i] for i in self.dfims_id[k]], mode=mode))
    
        # TAM
        tams = []
        for k in range(len(self.tams)):
            if k in self.predictors_id:
                tams.append(self.tams[k](dfims[k]))

        # Prediction
        predictions = []
        if mode == 3:
            for mode_ in range(mode):
                predictions.append(self.predictors[mode_]([tam[mode_:mode_+1] for tam in tams], x_size))
        else:
            predictions = self.predictors[mode](tams, x_size)
        return predictions

def build_model():
    return DFI(*extra_layer(resnet50_ppm()))


**DataSet Class**

In [73]:
import os
import torch
from torch.utils import data
from torchvision.transforms import functional as F
import cv2
import numpy as np
from PIL import Image

class ImageDataTest(data.Dataset):
    def __init__(self,input, test_mode=1, sal_mode='e'):
        if test_mode == 0:
            self.image_root = './data/HED-BSDS_PASCAL/HED-BSDS/test/'
            self.image_source = './data/HED-BSDS_PASCAL/HED-BSDS/test.lst'
        elif test_mode == 1:
            if sal_mode == 'e':
                self.image_root = './data/ECSSD/Imgs/'
                self.image_source = './data/ECSSD/test.lst'
            elif sal_mode == 'p':
                self.image_root = './data/PASCALS/Imgs/'
                self.image_source = './data/PASCALS/test.lst'
            elif sal_mode == 'd':
                self.image_root = './data/DUTOMRON/Imgs/'
                self.image_source = './data/DUTOMRON/test.lst'
            elif sal_mode == 'h':
                self.image_root = './data/HKU-IS/Imgs/'
                self.image_source = './data/HKU-IS/test.lst'
            elif sal_mode == 's':
                self.image_root = './data/SOD/Imgs/'
                self.image_source = './data/SOD/test.lst'
            elif sal_mode == 't':
                self.image_root = './data/DUTS-TE/Imgs/'
                self.image_source = './data/DUTS-TE/test.lst'
        elif test_mode == 2:
            self.image_root = './data/SK-LARGE/images/test/'
            self.image_source = './data/SK-LARGE/test.lst'
        elif test_mode == 3:
            self.image_root = './demo/images/'
            self.image_source = './demo/img.lst'

        elif test_mode == 4:
            self.image_root = ''

        if not test_mode ==4:
            with open(self.image_source, 'r') as f:
                self.image_list = [x.strip() for x in f.readlines()]
            self.image_num = len(self.image_list)
        else:
            self.image_list = [input]
            self.image_num = len(self.image_list)

    def __getitem__(self, item):
        image, im_size = load_image_test(os.path.join(self.image_root, self.image_list[item]))
        image = torch.Tensor(image)
        return {'image': image, 'name': self.image_list[item%self.image_num], 'size': im_size}

    def __len__(self):
        return self.image_num

def get_loader(input, test_mode=0, sal_mode='e', pin=False):
    dataset = ImageDataTest(input,test_mode=test_mode, sal_mode=sal_mode)
    data_loader = data.DataLoader(dataset=dataset, batch_size=1, shuffle=False, num_workers=1,
                                      pin_memory=pin)
    return data_loader

def load_image_test(pah):
    if not os.path.exists(pah):
        print('File Not Exists')
    im = cv2.imread(pah)
    in_ = np.array(im, dtype=np.float32)
    im_size = tuple(in_.shape[:2])
    in_ -= np.array((104.00699, 116.66877, 122.67892))
    in_ = in_.transpose((2,0,1))
    return in_, im_size

**SOLVER**

In [72]:
import torch
from torch.nn import utils, functional as F
from networks.dfi import build_model
import numpy as np
import os
import cv2
import ntpath

class Solver():
    def __init__(self, config):
        # self.data_loader = data_loader
        self.config = config
        self.net = build_model()
        print('Loading pre-trained model from %s...' % self.config.model)
        self.net = self.net.cpu()
        self.net.load_state_dict(torch.load(self.config.model))
        if self.config.cuda:
            self.net = self.net.cuda()
        torch.cuda.empty_cache()
        self.net.eval()

    def test(self, data_loader):
        test_mode = 4
        mode_name = ['edge', 'sal', 'skel']
        EPSILON = 1e-8
        img_num = len(data_loader)
        for i, data_batch in enumerate(data_loader):
            images, name, im_size = data_batch['image'], data_batch['name'][0], np.asarray(data_batch['size'])
            if test_mode == 0: # edge task
                images = images.numpy()[0].transpose((1,2,0))
                scale = [0.5, 1, 1.5, 2] # multi-scale testing as commonly done
                multi_fuse = np.zeros(im_size, np.float32)
                for k in range(0, len(scale)):
                    im_ = cv2.resize(images, None, fx=scale[k], fy=scale[k], interpolation=cv2.INTER_LINEAR)
                    im_ = im_.transpose((2, 0, 1))
                    im_ = torch.Tensor(im_[np.newaxis, ...])

                    with torch.no_grad():
                        if self.config.cuda:
                            im_ = im_.cuda()
                        preds = self.net(im_, mode=test_mode)
                        preds_i = []
                        for p in preds[1]:
                            preds_i.append(np.squeeze(torch.sigmoid(p).cpu().data.numpy()))
                        pred_fuse = np.squeeze(torch.sigmoid(preds[0]).cpu().data.numpy())
                        pred = (pred_fuse + sum(preds_i)) / (1.0 + len(preds_i))

                        pred = (pred - np.min(pred) + EPSILON) / (np.max(pred) - np.min(pred) + EPSILON)

                        pred = cv2.resize(pred, (im_size[1], im_size[0]), interpolation=cv2.INTER_LINEAR)
                        multi_fuse += pred

                multi_fuse /= len(scale)
                multi_fuse = 255 * (1 - multi_fuse)
                cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[test_mode] + '.png'), multi_fuse)

            elif test_mode == 1: # saliency task
                with torch.no_grad():
                    if self.config.cuda:
                        images = images.cuda()
                    preds = self.net(images, mode=test_mode)
                    pred = np.squeeze(torch.sigmoid(preds[0]).cpu().data.numpy())

                    multi_fuse = 255 * pred
                    cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[test_mode] + '.png'), multi_fuse)

            elif test_mode == 2: # skeleton task
                images = images.numpy()[0].transpose((1,2,0))
                scale = [0.5, 1, 1.5] # multi-scale testing as commonly done
                multi_fuse = np.zeros(im_size, np.float32)
                for k in range(0, len(scale)):
                    im_ = cv2.resize(images, None, fx=scale[k], fy=scale[k], interpolation=cv2.INTER_LINEAR)
                    im_ = im_.transpose((2, 0, 1))
                    im_ = torch.Tensor(im_[np.newaxis, ...])

                    with torch.no_grad():
                        if self.config.cuda:
                            im_ = im_.cuda()
                        preds = self.net(im_, mode=test_mode)
                        pred_fuse = np.squeeze(torch.sigmoid(preds[0]).cpu().data.numpy())

                        pred = pred_fuse
                        pred = (pred - np.min(pred) + EPSILON) / (np.max(pred) - np.min(pred) + EPSILON)

                        pred = cv2.resize(pred, (im_size[1], im_size[0]), interpolation=cv2.INTER_LINEAR)
                        multi_fuse += pred

                multi_fuse /= len(scale)
                multi_fuse = 255 * (1 - multi_fuse)
                cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[test_mode] + '.png'), multi_fuse)
            elif test_mode == 3: # all tasks
                with torch.no_grad():
                    if self.config.cuda:
                        images = images.cuda()
                    preds = self.net(images, mode=test_mode)
                    pred_edge = np.squeeze(torch.sigmoid(preds[0][0]).cpu().data.numpy())
                    pred_sal = np.squeeze(torch.sigmoid(preds[1][0]).cpu().data.numpy())
                    pred_skel = np.squeeze(torch.sigmoid(preds[2][0]).cpu().data.numpy())

                    pred_edge = (pred_edge - np.min(pred_edge) + EPSILON) / (np.max(pred_edge) - np.min(pred_edge) + EPSILON)
                    pred_skel = (pred_skel - np.min(pred_skel) + EPSILON) / (np.max(pred_skel) - np.min(pred_skel) + EPSILON)

                    cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[0] + '.png'), 255 * (1 - pred_edge))
                    cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[1] + '.png'), 255 * pred_sal)
                    cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[2] + '.png'), 255 * (1 - pred_skel))

            elif test_mode == 4:  # all tasks
                with torch.no_grad():
                    if self.config.cuda:
                        images = images.cuda()
                    preds = self.net(images, mode=3)
                    pred_edge = np.squeeze(torch.sigmoid(preds[0][0]).cpu().data.numpy())
                    pred_sal = np.squeeze(torch.sigmoid(preds[1][0]).cpu().data.numpy())
                    pred_skel = np.squeeze(torch.sigmoid(preds[2][0]).cpu().data.numpy())

                    pred_edge = (pred_edge - np.min(pred_edge) + EPSILON) / (np.max(pred_edge) - np.min(pred_edge) + EPSILON)
                    pred_skel = (pred_skel - np.min(pred_skel) + EPSILON) / (np.max(pred_skel) - np.min(pred_skel) + EPSILON)
                    # pred_sal = (pred_sal - np.min(pred_sal) + EPSILON) / (np.max(pred_sal) - np.min(pred_sal) + EPSILON)

                    name = ntpath.basename(name)
                    cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[0] + '.png'),
                                255 * (1 - pred_edge))
                    cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[1] + '.png'), 255 * pred_sal)
                    cv2.imwrite(os.path.join(self.config.test_fold, name[:-4] + '_' + mode_name[2] + '.png'),
                                255 * (1 - pred_skel))


        # print('Predicition of DFI Finished.')
        return pred_sal,pred_skel,pred_edge



In [68]:
class DFI_Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
def convert_arg_line_to_args(arg_line):
    for arg in arg_line.split():
        if not arg.strip():
            continue
        yield arg


In [74]:
from tqdm import tqdm
import os 

img_path = '/content/DFI/Everest/input_images/RO'
outpath = '/content/drive/My Drive/Everest/Ing_hasttags_RO_DFI'
image_files = os.listdir(img_path)
processed_files = os.listdir(outpath)
newfiles_list = []
for files in image_files:
    fn, ext = os.path.splitext(files)
    new_fn = fn+'_edge.png'
    if new_fn not in processed_files:
        newfiles_list.append(files)
print(len(newfiles_list),len(image_files))
config = DFI_Namespace(cuda=True, in_image='demo/images/ILSVRC2012_test_00002969.jpg',
                           model='pretrained/dfi.pth', sal_mode='e', 
                       test_fold=outpath,
                       test_mode=4)
# # if not os.path.exists(config.test_fold): os.mkdir(config.test_fold)
test = Solver(config)
for img in tqdm(newfiles_list):
    file = os.path.join(img_path,img)
    config.in_image = file
    data_loader = get_loader(config.in_image,test_mode=config.test_mode, sal_mode=config.sal_mode)
    DFI_sal,DFI_edge,DFI_skel = test.test(data_loader)



15754 17580
Loading pre-trained model from pretrained/dfi.pth...


Streaming output truncated to the last 5000 lines.
 84%|████████▍ | 13255/15754 [1:47:34<18:44,  2.22it/s]

 84%|████████▍ | 13256/15754 [1:47:34<19:15,  2.16it/s]

 84%|████████▍ | 13257/15754 [1:47:35<19:37,  2.12it/s]

 84%|████████▍ | 13258/15754 [1:47:35<19:07,  2.18it/s]

 84%|████████▍ | 13259/15754 [1:47:36<20:26,  2.03it/s]

 84%|████████▍ | 13260/15754 [1:47:36<17:38,  2.36it/s]

 84%|████████▍ | 13261/15754 [1:47:36<17:18,  2.40it/s]

 84%|████████▍ | 13262/15754 [1:47:37<16:39,  2.49it/s]

 84%|████████▍ | 13263/15754 [1:47:37<17:50,  2.33it/s]

 84%|████████▍ | 13264/15754 [1:47:38<19:43,  2.10it/s]

 84%|████████▍ | 13265/15754 [1:47:38<20:30,  2.02it/s]

 84%|████████▍ | 13266/15754 [1:47:39<19:51,  2.09it/s]

 84%|████████▍ | 13267/15754 [1:47:39<19:53,  2.08it/s]

 84%|████████▍ | 13268/15754 [1:47:40<20:16,  2.04it/s]

 84%|████████▍ | 13269/15754 [1:47:40<20:23,  2.03it/s]

 84%|████████▍ | 13270/15754 [1:47:41<20:25,  2.03it/s]

 84%|████████▍ | 13271/15754 [1:47:41